# Caminhos e pacotes

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 200

import os
import re
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import soundfile as sf  
from pydub import AudioSegment
import math  
import shutil
import numpy as np
from scipy.io import wavfile
import scipy
from IPython.display import Audio

import matplotlib.pyplot as plt
%matplotlib inline

###################################################################
#Para converter mp3 em wav
from pydub import AudioSegment
AudioSegment.converter = "C:\\ffmpeg\\bin\\ffmpeg.exe"
AudioSegment.ffmpeg = "C:\\ffmpeg\\bin\\ffmpeg.exe"
AudioSegment.ffprobe ="C:\\ffmpeg\\bin\\ffprobe.exe"
###################################################################


#Para montar MFCCs e Filter Banks
from python_speech_features import mfcc
from python_speech_features import logfbank

from time import gmtime, strftime

#Folder Inicial
path = os.getcwd()
pathin = path + '\\entrada\\'   
pathparcial = path + '\\parcial\\'
pathaux = path + '\\auxiliar\\'
pathout = path + '\\saida\\'

## Lendo Base de Treino

In [2]:
name = 'Teste'
df = joblib.load(pathparcial + name + 'V2.pkl')
df.reset_index(inplace = True)

df['mfccMean'] = np.nan
df['mfccMean'] = df['mfccMean'].astype(object)

for row in tqdm(range(0, df.shape[0])):  
    mfcc_feat = mfcc(df['signal'][row], df['rate'][row]) #Array de multiplas linhas e 13 colunas
    df.at[row, 'mfccMean'] = mfcc_feat.mean(axis=0) #Media por coluna
    
teste = df.drop(columns = ['signal'])
del df

100%|████████████████████████████████████████████████████████████████████████████████| 216/216 [00:17<00:00,  8.98it/s]


In [3]:
name = 'Train'
df = joblib.load(pathparcial + name + 'V2.pkl')
df.reset_index(inplace = True)

df['mfccMean'] = np.nan
df['mfccMean'] = df['mfccMean'].astype(object)

for row in tqdm(range(0, df.shape[0])):  
    mfcc_feat = mfcc(df['signal'][row], df['rate'][row]) #Array de multiplas linhas e 13 colunas
    df.at[row, 'mfccMean'] = mfcc_feat.mean(axis=0) #Media por coluna
    
train = df.drop(columns = ['signal'])
del df

100%|████████████████████████████████████████████████████████████████████████████████| 945/945 [01:15<00:00, 12.47it/s]


In [4]:
train0 = train.copy()
teste0 = teste.copy()

## Construindo o Modelo

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.utils import parallel_backend

## Modelo 1: Genero

In [6]:
id = 'file'
id2 = 'cantor'
y = 'masculino'
x = 'mfccMean'

train = train[[id, id2, y, x]]
teste = teste[[id, id2, y, x]]

#Treino
trainMFCC = pd.DataFrame(train[x].values.tolist(),
                         columns=['mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 
                                  'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13'])
train = pd.concat([train, trainMFCC], axis=1)
train.drop(columns = [x], inplace = True)

#Teste
testeMFCC = pd.DataFrame(teste[x].values.tolist(),
                         columns=['mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 
                                  'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13'])
teste = pd.concat([teste, testeMFCC], axis=1)
teste.drop(columns = [x], inplace = True)

varsmod = ['mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 
                                  'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13']

### Logistica

In [7]:
nomemodelo = 'Logistica'

seed = 123

parameters = {
    #'max_iter': [5000],
    'C': [.01, .1, 1],
    'penalty': ['l1'],
    'fit_intercept': [True],
    'class_weight': ['balanced', None],
    #'clf__solver': ['lbfgs'],
    'random_state': [seed]
}

estimator = LogisticRegression()

cv = RepeatedStratifiedKFold(n_splits = 3, n_repeats = 1, random_state = seed)

gs = GridSearchCV(estimator = estimator, param_grid = parameters, cv = cv,
                  scoring = 'roc_auc', n_jobs = 3, verbose = 1, refit = True, return_train_score = True)

In [8]:
#Rodando GridSearch
print(strftime('%Y-%m-%d %H:%M:%S', gmtime()))

with parallel_backend('multiprocessing'):
    gs.fit(train[varsmod], train[y])

joblib.dump(gs, pathaux + 'Modelo ' + nomemodelo + '.pkl')

print(strftime('%Y-%m-%d %H:%M:%S', gmtime()))

2019-09-19 03:39:55
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=3)]: Using backend MultiprocessingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  18 out of  18 | elapsed:    4.7s finished
C:\Users\laran\Anaconda3\envs\transc\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


2019-09-19 03:40:00


In [9]:
gs.best_score_

0.9719518785599636

In [10]:
gs.best_params_

{'C': 1,
 'class_weight': None,
 'fit_intercept': True,
 'penalty': 'l1',
 'random_state': 123}

In [11]:
cvresults = pd.DataFrame(gs.cv_results_)[[
 'mean_test_score',
 'mean_train_score',
 'param_C',
 'param_penalty',
 'param_fit_intercept',
 'param_class_weight',
 'std_test_score',
 'std_train_score']]
ha = list(cvresults)
ha = [w.replace('param_', '') for w in ha]
cvresults.columns = ha
cvresults.to_excel(pathout + 'Modelo ' + nomemodelo + ' Resultados GridSearch.xlsx', 
                   encoding = 'latin1', index = False, engine = 'openpyxl')

In [12]:
cvresults

,mean_test_score,mean_train_score,C,penalty,fit_intercept,class_weight,std_test_score,std_train_score
0,0.939906,0.945907,0.01,l1,True,balanced,0.009230,0.007218
1,0.939960,0.945920,0.01,l1,True,None,0.009151,0.007223
2,0.952734,0.957796,0.1,l1,True,balanced,0.014742,0.006431
3,0.952774,0.957843,0.1,l1,True,None,0.014820,0.006473
4,0.971898,0.976743,1,l1,True,balanced,0.012034,0.004140
5,0.971952,0.976827,1,l1,True,None,0.012015,0.004120


#### Features

In [13]:
def Feature_Importance(mod, nomemodelo, X, y):
    many = ['Linear SVM', 'Logistica', 'Naive Bayes']

    k = mod.best_estimator_
    featurenames = list(X)
    
    if nomemodelo in many:
        #Logistica, SVM, Naive Bayes
        Features = pd.DataFrame(k.coef_.tolist())
        Features.columns = featurenames
        if len(y.unique()) > 2:
            Features.index = sorted(y.unique())
    else:
        #Arvores
        Features = pd.DataFrame({'features': featurenames, 'value': k.feature_importances_.tolist()})
    
    print(Features)
    Features.to_excel(pathout + 'Features ' + nomemodelo + '.xlsx', encoding = 'latin1', index = True)
    
Feature_Importance(mod = gs, nomemodelo = nomemodelo, X = train[varsmod], y = train[y])

     mfcc1     mfcc2     mfcc3     mfcc4     mfcc5     mfcc6     mfcc7  \
0  1.22474 -0.340621  0.174583 -0.422942 -0.162481 -0.102888 -0.168227   

      mfcc8     mfcc9    mfcc10    mfcc11    mfcc12    mfcc13  
0  0.215698  0.134096  0.067047 -0.038332  0.153883 -0.361664  


#### Preditos

In [14]:
predtr = gs.best_estimator_.predict(train[varsmod])
predte = gs.best_estimator_.predict(teste[varsmod])

predtrP = pd.DataFrame(gs.best_estimator_.predict_proba(train[varsmod]))
predteP = pd.DataFrame(gs.best_estimator_.predict_proba(teste[varsmod]))

predtrP = predtrP[predtrP.columns[1]]
predteP = predteP[predteP.columns[1]]

In [15]:
print('Auc')
print(roc_auc_score(y_true = train[y], y_score = predtrP))
print()
print('Acurácia')
print(accuracy_score(y_true = train[y], y_pred = predtr))
print()
print('F1')
print(f1_score(y_true = train[y], y_pred = predtr))
print()
print('Precision')
print(precision_score(y_true = train[y], y_pred = predtr))
print()
print('Recall')
print(recall_score(y_true = train[y], y_pred = predtr))
print()

Auc
0.9788196715436375

Acurácia
0.928042328042328

F1
0.9282700421940928

Precision
0.9147609147609148

Recall
0.9421841541755889



In [16]:
print('Auc')
print(roc_auc_score(y_true = teste[y], y_score = predteP))
print()
print('Acurácia')
print(accuracy_score(y_true = teste[y], y_pred = predte))
print()
print('F1')
print(f1_score(y_true = teste[y], y_pred = predte))
print()
print('Precision')
print(precision_score(y_true = teste[y], y_pred = predte))
print()
print('Recall')
print(recall_score(y_true = teste[y], y_pred = predte))
print()

Auc
0.7462504359958144

Acurácia
0.6990740740740741

F1
0.6632124352331606

Precision
0.6464646464646465

Recall
0.6808510638297872



In [17]:
name = 'Treino'
fim = train[[id, id2, y]]
fim['pred'] = predtr

fim['Acertou'] = np.where(fim[y] == fim['pred'], 1, 0)
fim.to_excel(pathout + 'Modelo ' + nomemodelo + ' Pred ' + name + '.xlsx', encoding = 'latin1', index = False)

del fim

name = 'Teste'
fim = teste[[id, id2, y]]
fim['pred'] = predte

fim['Acertou'] = np.where(fim[y] == fim['pred'], 1, 0)
fim.to_excel(pathout + 'Modelo ' + nomemodelo + ' Pred ' + name + '.xlsx', encoding = 'latin1', index = False)

C:\Users\laran\Anaconda3\envs\transc\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\laran\Anaconda3\envs\transc\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\laran\Anaconda3\envs\transc\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u